## Importing packages

In [3]:
import numpy
import sys
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


## Loading the text file

In [4]:
# load ascii text and covert to lowercase
file = open('C://Users/user/wonderland.txt', encoding = 'utf8')
raw_text = file.read()    #you need to read further characters as well
raw_text = raw_text.lower()

# Creating mapping of unique chars to integers

In [5]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

## Number of characters and number of vocabulary

In [6]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  163816
Total Vocab:  61


## Having removed first hundred chars, we have total chars - 100 patterns, which means one pattern for each remaining char

In [7]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  163716


## Transforming data to be suitable for keras

In [8]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

## Defining our LSTM model

In [9]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

## Model weights and checkpointing to note down the loss after each epoch

In [10]:
filepath="C://Users/user/char_LSTM_saved_models/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

## Fitting the model

In [ ]:
## Run atleast 25 epochs
model.fit(X, y, epochs=5, batch_size=128, callbacks=callbacks_list)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
101376/163716 [=================>............] - ETA: 25:48 - loss: 3.0562

## Selecting the one with less loss

In [11]:
filename = "char_LSTM_saved_models/weights-improvement-05-2.6140.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

## Reverse mapping to convert the integers back to characters so that we can understand the predictions.

In [12]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

## Preidiction of text

In [21]:
# pick a random seed to select a random starting point in the text
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
#print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters, we generate 10 characters here
for i in range(10):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\n")

" ed lately,
that alice had begun to think that very few things indeed were really
impossible.

there  "
toe toot t

